In [6]:
import requests
#import pandas as pd


from pymongo import MongoClient

In [12]:
def transform_and_load_data(json_file: list) -> dict:
    """
    Transforms a JSON file into a new dictionary containing all relevant Pokémon data for our work.

    Args:
        json_file (list): List containing the JSON response file from the API.
        

    Returns:
        dict: Dictionary containing all relevant information from Pokémon.
    """
    # Create a MongoDB client
    client = MongoClient()

    # Access the 'genetic_algo' database
    db = client.genetic_algo

    # Access the 'pokemons' collection within the 'genetic_algo' database
    collection = db.pokemons

    for pokemon in json_file:
        
         # Stores relevant data for each pokemon in a dict
        pokemon_data = dict()
        
        # Get name
        pokemon_data['name'] = pokemon.get('names', {}).get('English', None)
        
        # Get stats safely
        stats = pokemon.get('stats', {})
        if stats:  # Check if stats is not None
            pokemon_data['stamina'] = stats.get('stamina', None)
            pokemon_data['attack'] = stats.get('attack', None)
            pokemon_data['defense'] = stats.get('defense', None)
        else:
            pokemon_data['stamina'] = None
            pokemon_data['attack'] = None
            pokemon_data['defense'] = None

        # Get Pokemon's primaryType
        pokemon_data['primaryType'] = pokemon.get('primaryType').get('names').get('English')
        
        # Get Pokemon's secondaryType
        try:
            pokemon_data['secondaryType'] = pokemon.get('secondaryType').get('names').get('English')
        except AttributeError:
            pokemon_data['secondaryType'] = None

        special_moves = ["quickMoves", "cinematicMoves", "eliteQuickMoves", "eliteCinematicMoves"]

        # Get special moves data
        for move in special_moves:
            type_of_move = pokemon.get(move, {})

            if type_of_move:
                pokemon_data[move] = dict()
                aux_vec = []

                try:
                    for move_name, move_data in type_of_move.items():
                        aux_dict = dict()
                        
                        # Get move name
                        move_aux = move_data.get('names').get('English')
                        aux_dict['move_name'] = move_aux
                        
                        # Get move power
                        power = move_data.get('power')
                        aux_dict['move_power'] = power

                        # Get move energy
                        energy = move_data.get('energy')
                        aux_dict['move_energy'] = energy

                        # Get move cooldown duration
                        duration_ms = move_data.get('durationMs')
                        aux_dict['move_cd'] = duration_ms

                        # Get move combat stats
                        combat = move_data.get('combat', {})    
                        aux_dict['move_combat_stats'] = combat

                        aux_vec.append(aux_dict)

                    for index, info_pokemon_move in enumerate(aux_vec):
                        pokemon_data[move][str(index + 1)] = info_pokemon_move
                except AttributeError:
                    continue
        collection.insert_one(pokemon_data)
    

try:
    r = requests.get('https://pokemon-go-api.github.io/pokemon-go-api/api/pokedex.json')
    r.raise_for_status()
except requests.exceptions.RequestException as e:
    print('Erro:', e)  

# Get content of the response
r_content = r.json()

# Transforms the json file
pokemon_data = transform_and_load_data(r_content)



